# Building a Spam Filter with Naive Bayes

In this project we are going to use the Naive Bayes algorithm to build a spam filter for SMS messages. Our first task is to "teach" the computer how to classify messages through this algorithm and we'll use a dataset of 5,572 SMS messages that are already classified by humans.

The data set can be downloaded from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The SMS Spam Collection is a public set of SMS labeled messages that have been collected for mobile phone spam research. It has one collection composed by 5,572 English, real messages, tagged according being legitimate (ham) or spam.

Let's take a look at the data.

## Exploring the Data

We'll start reading the data in a dataframe.

In [1]:
#Importing the library
import pandas as pd

#Reading the data
sms_labels = pd.read_csv("SMSSpamCollection", sep="\t", header=None, names=["Label", "SMS"])

#Exploring the dimension of the data
sms_labels.shape

(5572, 2)

In [2]:
#Showing the first 4 rows
sms_labels.head(4)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...


In [3]:
#Finding the percentage of spam and ham messages
sms_labels["Label"].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

About the 87% of the SMS are ham messages, (ham means non-spam), while the remaining 13% are spam messages. This seems to be reasonable since most messages that people receive are ham.

## Training and Test Set

Before creating the spam filter, it's very helpful to first think of a way of testing how well it works. Once our spam filter is done, we'll need how good it is with classifying new messages. To test the filter, we're first going to split our dataset into two categories:

* A **training set**, which we'll use to train the compuer how to classify messages.
* A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to take 80% of our data set for training, and 20% for testing because we want to train our algorithm on as much data as possible, but we also want enough test data. The data set has 5,572 messages, which means:

* The training set will have 4,458 messages.
* The test set will have 1,114 messages.

All the messages are already classified by a human. When the spam filter is ready we're going to treat the test set messages as new and the filter will classify them. Once we have the results, we can compare the algorithm results with that done by human, and we'll see how good the spam filter really is.

For this project we want our spam filter to reach at least 80% of accuracy.

In [4]:
#Randomize the dataset
data_random = sms_labels.sample(frac=1, random_state=1)

#Creating the training and test sets
training_set = data_random[:round(sms_labels.shape[0]*0.8)].reset_index(drop=True)
test_set = data_random[round(sms_labels.shape[0]*0.8):].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [5]:
#Finding percentage for the training set
training_set["Label"].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [6]:
#Finding percentage for the test set
test_set["Label"].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The percentages look good, they are very similar to the percentages of the entire dataset.

## Letter Case and Punctuation

We'll first need to perform a bit of data cleaning to bring the data in a form that will allow us to extract easily the information we need. We want to bring the data from this format:

![](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_1.png)

to this format:

![](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_2.png)

Let's start removing all the punctuation and transforming every letter to lower case.

In [7]:
#Showing data before cleaning
training_set.head(4)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.


In [8]:
#Removing the punctuation
training_set["SMS"] = training_set["SMS"].str.replace('\W', ' ')

#Setting all words to lower case
training_set["SMS"] = training_set["SMS"].str.lower()

#Showing data after cleaning
training_set.head(4)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent


## Creating the Vocabulary

With the exception of the *Label* column, every other column in the table will represents a unique word present in messages. Let's create a list with all these unique words: the vocabulary.

In [9]:
#Transforming the column into a list
training_set["SMS"] = training_set["SMS"].str.split()

#Creating the vocabulary
vocabulary = []
for element in training_set["SMS"]:
    for word in element:
        if word not in vocabulary:
            vocabulary.append(word)

#Showing the length of the vocabulary            
len(vocabulary)

7783

## The Final Training Set

On the previous screen, we managed to create the vocabulary for our messages in the training set. Now we're going to use the vocabulary to make the data transformation we need, eventually creating a new dataframe.

In [10]:
#Creating the dictionary
word_counts_per_sms = {unique_word: [0] * len(training_set["SMS"]) for unique_word in vocabulary}

#Populating the dictionary
for index, sms in enumerate(training_set["SMS"]):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
#Creating the dataframe
word_counts = pd.DataFrame(data=word_counts_per_sms)

#Showing the dataframe
word_counts.head(4)

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#Concatenating the two dataframes
training_set_final = pd.concat([training_set, word_counts], axis=1)

#Showing the final dataframe
training_set_final.head(4)

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

To use the Naive Bayes algorithm we'll need to know the probability values of the two equations below to be able to classify messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

And to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In particular:

* P(Spam)
* P(Ham)
* N<sub>Spam</sub>
* N<sub>Ham</sub>
* N<sub>Vocabulary</sub>

have the same values, so let's start calculate them.

We'll also use Laplace smoothing and set $\alpha = 1$.

In [12]:
#Isolating spam and ham messages
spam = training_set_final[training_set_final["Label"] == "spam"]
ham = training_set_final[training_set_final["Label"] == "ham"]

#Computing P(Spam) and P(Ham)
p_spam = len(spam) / len(training_set_final)
p_ham = len(ham) / len(training_set_final)

#Computing N(Spam) and N(Ham)
n_spam = spam["SMS"].apply(len).sum()
n_ham = ham["SMS"].apply(len).sum()

#Computing N(Vocabulary)
n_vocabulary = len(vocabulary)

#Laplace smoothing
alpha = 1

## Calculating Parameters

We had calculate all the constant terms of our equation. Now we'll calculate the parameters P(wi|Spam) and P(wi|Ham) using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [13]:
#Creating the dictionaries for spam and ham messages
spam_vocabulary = {key: 0 for key in vocabulary}
ham_vocabulary = {key: 0 for key in vocabulary}

#Computing P(wi|Spam) and P(wi|Ham)
for word in vocabulary:
    p_word_spam = (spam[word].sum() + alpha) / (n_spam + alpha * n_vocabulary)
    p_word_ham = (ham[word].sum() + alpha) / (n_ham + alpha * n_vocabulary)
    spam_vocabulary[word] = p_word_spam
    ham_vocabulary[word] = p_word_ham

## Classifying a New Message

Now we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message(w1, w2, ..., wn)
* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
* Compares the values and returns as output if the message is spam or ham.

In [14]:
#Importing the library
import re

#Creating the function to classify messages
def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in spam_vocabulary:
            p_spam_given_message *= spam_vocabulary[word]
        if word in ham_vocabulary:
            p_ham_given_message *= ham_vocabulary[word]
        
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Now let's test our function with both a spam and a ham input.

In [15]:
#Testing spam message
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [16]:
#Testing ham message
classify('Sounds good, Tom, then see u there')

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

We'll now try to determine how well the spam filter does on our test set of 1,114 messages. Let's make some changes to the classify() function in order to calculate the accuracy of our spam filter.

In [17]:
#Modifying the function
def classify_test_set(message):
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message:
        if word in spam_vocabulary:
            p_spam_given_message *= spam_vocabulary[word]
        if word in ham_vocabulary:
            p_ham_given_message *= ham_vocabulary[word]
        
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    
#Creating the column of the prediction
test_set["Predicted"] = test_set["SMS"].apply(classify_test_set)

#Showing the changing
test_set.head(4)

,Label,SMS,Predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham


In [19]:
#Measuring the accuracy of the spam filter
correct = 0
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row["Label"] == row["Predicted"]:
        correct += 1
        
print("Correct: ", correct)
print("Incorrect: ", total - correct)
print("Accuracy: ", correct / total)

Correct:  1100
Incorrect:  14
Accuracy:  0.9874326750448833


The result is very good, the accuracy is above 98.7%. Our spam filter analyzed 1,114 messages that it hasn't seen in the training session and classified 1,100 correctly.

## Conclusion

In this project, we managed to build a spam filter using the Naive Bayes algorithm. Our initial goal was an accuracy of over 80%, and we managed to do way better than that with an accuracy of 98.74%. To do even better, we could:

* Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusion.
* Make the algorithm more complex considering letter case.